# Benchmark model - Pastdue180 - Exploration, calibration, test

This notebook uses as inputs the outputs from the pipeline ('05_preproc_pipeline_1.ipynb' notebook) and performs model calibration and general exploration for the transaction credit events prediction.

In [1]:
import pandas as pd
import numpy as np
import pickle
from os import environ

from sklearn.model_selection import cross_val_predict, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from scripts_viz.visualization_utils import *
from scripts_ml.models_utils import *

from bokeh.io import show, output_notebook
output_notebook()

import datetime as dt

Loading BokehJS ...

### TRAIN Data import

In [2]:
#importing data
user=environ["USERNAME"]
prefix = 'shuffle_p180_'
trainfile = '_traindata'
testfile = '_testdata'
postfix = '_19072_750'

#from home
datafolder = ".."+"/data/preproc_traintest/"

[X_train, y_train, feature_labels] = pd.read_pickle(datafolder+prefix+trainfile+postfix+'.pkl') 
[X_test, y_test, feature_labels] = pd.read_pickle(datafolder+prefix+testfile+postfix+'.pkl') 

In [3]:
X_train_df = pd.DataFrame(X_train, columns=feature_labels)
X_train_df.head()

,has_purchase,dd_value_date,cd_lent_c,cd_repaid_c,cd_impaired1_c,cd_pastdue90_c,cd_trend_a,c_lent_c,c_repaid_c,c_impaired1_c,...,d_pastdue90_c,d_trend_a,d_we_payment_share,invoice_amount,purchase_amount,currency_Britisches Pfund,currency_Euro,currency_Schweizer Franken,currency_US-Dollar,invoice_date
0,0.478098,-0.145273,-0.528606,-0.493910,-0.17567,-0.214037,-1.385604,-1.009960,-0.893308,-0.554907,...,-0.215463,-1.378706,0.117320,0.967374,0.678658,0.0,0.0,1.0,0.0,-1.870522
1,0.478098,-0.145273,-0.302102,-0.493910,-0.17567,-0.214037,-0.960531,-0.938481,-0.893308,-0.554907,...,-0.215463,-0.950582,0.117320,2.014475,0.905137,0.0,1.0,0.0,0.0,-0.606708
2,0.478098,-0.145273,-0.457419,-0.368922,-0.17567,-0.214037,0.997265,-0.922398,-0.788296,-0.554907,...,-0.215463,1.021271,-0.192621,-0.012316,0.466758,0.0,0.0,1.0,0.0,0.315048
3,0.478098,-0.145273,-0.502720,-0.460580,-0.17567,-0.214037,1.624837,-0.989410,-0.861077,-0.554907,...,-0.215463,1.653350,-0.192621,-0.829188,0.290076,0.0,0.0,1.0,0.0,0.675109
4,0.478098,-0.145273,-0.522134,-0.443915,-0.17567,-0.214037,1.315468,0.274891,0.239989,0.079527,...,-0.215463,1.341759,-0.192621,1.362819,0.764189,0.0,0.0,1.0,0.0,0.523883


In [4]:
X_train_df.columns

Index(['has_purchase', 'dd_value_date', 'cd_lent_c', 'cd_repaid_c',
       'cd_impaired1_c', 'cd_pastdue90_c', 'cd_trend_a', 'c_lent_c',
       'c_repaid_c', 'c_impaired1_c', 'c_pastdue90_c', 'c_trend_a',
       'cd_lent_c', 'd_repaid_c', 'd_impaired1_c', 'd_pastdue90_c',
       'd_trend_a', 'd_we_payment_share', 'invoice_amount', 'purchase_amount',
       'currency_Britisches Pfund', 'currency_Euro',
       'currency_Schweizer Franken', 'currency_US-Dollar', 'invoice_date'],
      dtype='object')

In [6]:
X_train_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
has_purchase,46095.0,-2.067704e-08,1.000011,-2.091622,0.478098,0.478098,0.478098,0.478098
dd_value_date,46095.0,3.199733e-09,1.000011,-0.145273,-0.145273,-0.145273,-0.145273,85.601524
cd_lent_c,46095.0,1.108227e-09,1.000011,-0.567435,-0.528606,-0.431533,-0.017355,4.393315
cd_repaid_c,46095.0,-1.870823e-08,1.000011,-0.493910,-0.493910,-0.418917,-0.093949,4.377090
cd_impaired1_c,46095.0,-1.428856e-10,1.000011,-0.175670,-0.175670,-0.175670,-0.175670,8.903720
cd_pastdue90_c,46095.0,-4.453377e-09,1.000011,-0.214037,-0.214037,-0.214037,-0.214037,6.398803
cd_trend_a,46095.0,-1.419782e-08,1.000011,-3.920882,-0.678948,-0.036710,0.712322,3.604046
c_lent_c,46095.0,-2.698500e-08,1.000011,-1.037659,-0.801775,-0.315712,0.566171,3.869435
c_repaid_c,46095.0,1.823345e-08,1.000011,-0.893308,-0.788296,-0.336017,0.435457,4.005999
c_impaired1_c,46095.0,1.572906e-08,1.000011,-0.554907,-0.554907,-0.343429,0.079527,4.806320


### SGD Classifier

In [7]:
#Linear model Stochastic Gradient Descent
sgd_clf = SGDClassifier(random_state=42, max_iter=250, loss='log', tol=0.0001) 
sgd_clf.fit(X_train, y_train)
sgd_clf_diag = model_diag(sgd_clf, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.771
Confusion matrix: 
[[0.99787 0.00213]
 [0.06398 0.00229]]


In [8]:
y_score_sgd = sgd_clf.predict(X_test)
cm = confusion_matrix(y_test, y_score_sgd)
print(cm)
sgd_test_diag = model_oostest(sgd_clf,X_test, y_test)
sgd_test_auc = plot_rocs([sgd_clf_diag, sgd_test_diag], p_width=600, p_height=600, model_appendix=['SGD - 5folds','SGD - test'])
show(sgd_test_auc)

[[10769    18]
 [  704    31]]
[[10769    18]
 [  704    31]]
AUC 0.786


In [9]:
#save_sk_model(sgd_clf, "../data/models/", 'sgd', 'benchmark')

In [10]:
# max Number of iterations
max_iter = [100, 180, 200, 250, 300, 320, 350] 
# learning rate
learning_rate = ['constant', 'optimal', 'adaptive']
#eta
eta0 = [0.00001, 0.0001, 0.001, 0.01]
#loss
loss = ['log', 'hinge', 'perceptron']

sgd_random_grid = {'max_iter': max_iter,
               'learning_rate': learning_rate,
                  'eta0': eta0,
                  'loss':loss}

sgd = SGDClassifier(tol=0.0001)

sgd_rsearch = RandomizedSearchCV(estimator = sgd, param_distributions = sgd_random_grid, n_iter = 200, cv = 3, verbose=2, random_state=42, n_jobs =7)
# Fit the random search model
sgd_rsearch.fit(X_train, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:    3.2s
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed:    9.8s
[Parallel(n_jobs=7)]: Done 351 tasks      | elapsed:   21.3s
[Parallel(n_jobs=7)]: Done 600 out of 600 | elapsed:   37.4s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=0.0001,
       validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=200, n_jobs=7,
          param_distributions={'max_iter': [100, 180, 200, 250, 300, 320, 350], 'learning_rate': ['constant', 'optimal', 'adaptive'], 'eta0': [1e-05, 0.0001, 0.001, 0.01], 'loss': ['log', 'hinge', 'perceptron']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [11]:
sgd_rsearch.best_params_

{'max_iter': 100, 'loss': 'log', 'learning_rate': 'optimal', 'eta0': 0.001}

In [12]:
#Linear model Stochastic Gradient Descent
sgd_rs = SGDClassifier(random_state=42, max_iter=100, loss='log', learning_rate='optimal', eta0=0.001, tol=0.0001) 
sgd_rs.fit(X_train, y_train)
sgd_rs_diag = model_diag(sgd_rs, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.771
Confusion matrix: 
[[0.99787 0.00213]
 [0.06398 0.00229]]


In [13]:
y_score_sgd_rs = sgd_rs.predict(X_test)
cm = confusion_matrix(y_test, y_score_sgd_rs)
print(cm)
sgd_test_diag_rs = model_oostest(sgd_rs,X_test, y_test)
sgd_test_auc_rs = plot_rocs([sgd_rs_diag, sgd_test_diag_rs], p_width=600, p_height=600, model_appendix=['SGD - 5folds','SGD - test'])
show(sgd_test_auc_rs)

[[10769    18]
 [  704    31]]
[[10769    18]
 [  704    31]]
AUC 0.786


### Random Forest Classifier

In [14]:
#random forest - benchmark
n_estimators = 200
max_leaf_nodes = 40
rf_clf = RandomForestClassifier(random_state=42,
                               n_estimators=n_estimators,
                               max_leaf_nodes=max_leaf_nodes,
                               class_weight="balanced",
                               n_jobs=7)

rf_clf.fit(X_train, y_train)
rf_clf_diag = model_diag(rf_clf, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.957
Confusion matrix: 
[[0.8662  0.1338 ]
 [0.00618 0.0601 ]]


In [15]:
y_score_rf = rf_clf.predict(X_test)
cm = confusion_matrix(y_test, y_score_rf)
print(cm)
rf_test_diag = model_oostest(rf_clf,X_test, y_test)
rf_test_auc = plot_rocs([rf_clf_diag, rf_test_diag], p_width=600, p_height=600, model_appendix=['RF - 5folds','RF - test'])
show(rf_test_auc)

[[9276 1511]
 [  62  673]]
[[9276 1511]
 [  62  673]]
AUC 0.959


In [16]:
# Number of trees in random forest
n_estimators = [150, 180, 200, 250, 280, 300, 350] 
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 10, 20]
# Maximum number of levels in tree
max_depth = [100,200, None]
#Max leaf nodes
max_leaf_nodes = [10,20,40,60]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
              'max_leaf_nodes':max_leaf_nodes}
print(random_grid)

{'n_estimators': [150, 180, 200, 250, 280, 300, 350], 'max_features': ['auto', 'sqrt', 10, 20], 'max_depth': [100, 200, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'max_leaf_nodes': [10, 20, 40, 60]}


In [17]:
rf = RandomForestClassifier() #base model
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 300, cv = 3, verbose=2, random_state=42, n_jobs =7)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 300 candidates, totalling 900 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:  1.1min
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed:  6.0min
[Parallel(n_jobs=7)]: Done 351 tasks      | elapsed: 13.7min
[Parallel(n_jobs=7)]: Done 634 tasks      | elapsed: 25.4min
[Parallel(n_jobs=7)]: Done 900 out of 900 | elapsed: 37.2min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=300, n_jobs=7,
          param_distributions={'n_estimators': [150, 180, 200, 250, 280, 300, 350], 'max_features': ['auto', 'sqrt', 10, 20], 'max_depth': [100, 200, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'max_leaf_nodes': [10, 20, 40, 60]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [18]:
rf_random.best_params_

{'n_estimators': 350,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_leaf_nodes': 60,
 'max_features': 20,
 'max_depth': None,
 'bootstrap': True}

In [19]:
#first randomized search_imp
rs1 = {'n_estimators': 280,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_leaf_nodes':60,
 'max_depth': 100,
 'bootstrap': True}

#second randomized search_imp
rs2 = {'n_estimators': 150,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_leaf_nodes': 60,
 'max_features': 20,
 'max_depth': 100,
 'bootstrap': False}

#third randomized search_p90
rs3 = {'n_estimators': 150,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_leaf_nodes': 60,
 'max_features': 20,
 'max_depth': 100,
 'bootstrap': False}

#third randomized search_p180
rs6 = {'n_estimators': 350,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_leaf_nodes': 60,
 'max_features': 20,
 'max_depth': None,
 'bootstrap': True}

In [20]:
#random_forest from rand
n_estimators=rs6['n_estimators']
min_samples_split=rs6['min_samples_split']
min_samples_leaf=rs6['min_samples_leaf']
max_leaf_nodes=rs6['max_leaf_nodes']
max_features=rs6['max_features']
max_depth=rs6['max_depth']
bootstrap=rs6['bootstrap']




rf_rs = RandomForestClassifier(random_state=42,
                               n_estimators=n_estimators,
                                max_features=max_features,
                                min_samples_split=min_samples_split,
                               min_samples_leaf=min_samples_leaf,
                               class_weight="balanced", bootstrap=bootstrap,
                               n_jobs=7)
rf_rs.fit(X_train, y_train)
rf_rs_diag = model_diag(rf_rs, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.983
Confusion matrix: 
[[0.98658 0.01342]
 [0.01383 0.05244]]


In [21]:
y_score_rf_rs = rf_rs.predict(X_test)
cm = confusion_matrix(y_test, y_score_rf_rs)
print(cm)
rf_rs_test_diag = model_oostest(rf_rs,X_test, y_test)
rf_rs_test_auc = plot_rocs([rf_rs_diag, rf_rs_test_diag], p_width=600, p_height=600, model_appendix=['RF - 5folds','RF - test'])
show(rf_rs_test_auc)

[[10643   144]
 [  130   605]]
[[10643   144]
 [  130   605]]
AUC 0.986


In [22]:
save_sk_model(rf_rs, "../data/models/", 'rf_p180', 'benchmark')

Saving model to ../data/models/benchmark_rf_p180_190710_1257.pkl


### Training Performance Comparison

In [23]:
rf_sgd_roc = plot_rocs([sgd_clf_diag, rf_rs_diag], p_width=600, p_height=600, model_appendix=['SGD', 'Random Forest'])
show(rf_sgd_roc)

### Test Performance Comparison

In [24]:
rf_sgd_roc = plot_rocs([sgd_test_diag, rf_rs_test_diag], p_width=600, p_height=600, model_appendix=['SGD', 'Random Forest'])
show(rf_sgd_roc)

## REPEATING THE EXPERIMENT IN TIME MODE

In [25]:
prefix = 'time_2018-02-20_p180_'
postfix = '_190710_745'
[X_train, y_train, feature_labels] = pd.read_pickle(datafolder+prefix+trainfile+postfix+'.pkl') 
[X_test, y_test, feature_labels] = pd.read_pickle(datafolder+prefix+testfile+postfix+'.pkl') 

### SGD Classifier

In [26]:
#Linear model Stochastic Gradient Descent
sgd_clf = SGDClassifier(random_state=42, max_iter=250, loss='log', tol=0.0001) 
sgd_clf.fit(X_train, y_train)
sgd_clf_diag = model_diag(sgd_clf, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.685
Confusion matrix: 
[[0.99211 0.00789]
 [0.08292 0.00914]]


In [27]:
y_score_sgd = sgd_clf.predict(X_test)
cm = confusion_matrix(y_test, y_score_sgd)
print(cm)
sgd_test_diag = model_oostest(sgd_clf,X_test, y_test)
sgd_test_auc = plot_rocs([sgd_clf_diag, sgd_test_diag], p_width=600, p_height=600, model_appendix=['SGD - 5folds','SGD - test'])
show(sgd_test_auc)

[[14805   681]
 [   52     2]]
[[14805   681]
 [   52     2]]
AUC 0.692


In [28]:
#save_sk_model(sgd_clf, "../data/models/", 'sgd', 'benchmark')

In [29]:
# max Number of iterations
max_iter = [100, 180, 200, 250, 300, 320, 350] 
# learning rate
learning_rate = ['constant', 'optimal', 'adaptive']
#eta
eta0 = [0.00001, 0.0001, 0.001, 0.01]
#loss
loss = ['log', 'hinge', 'perceptron']

sgd_random_grid = {'max_iter': max_iter,
               'learning_rate': learning_rate,
                  'eta0': eta0,
                  'loss':loss}

sgd = SGDClassifier(tol=0.0001)

sgd_rsearch = RandomizedSearchCV(estimator = sgd, param_distributions = sgd_random_grid, n_iter = 200, cv = 3, verbose=2, random_state=42, n_jobs =7)
# Fit the random search model
sgd_rsearch.fit(X_train, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:    1.8s
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed:    8.0s
[Parallel(n_jobs=7)]: Done 351 tasks      | elapsed:   19.2s
[Parallel(n_jobs=7)]: Done 600 out of 600 | elapsed:   34.1s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=0.0001,
       validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=200, n_jobs=7,
          param_distributions={'max_iter': [100, 180, 200, 250, 300, 320, 350], 'learning_rate': ['constant', 'optimal', 'adaptive'], 'eta0': [1e-05, 0.0001, 0.001, 0.01], 'loss': ['log', 'hinge', 'perceptron']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [30]:
sgd_rsearch.best_params_

{'max_iter': 200, 'loss': 'log', 'learning_rate': 'constant', 'eta0': 0.0001}

In [31]:
#Linear model Stochastic Gradient Descent
sgd_rs = SGDClassifier(random_state=42, max_iter=200, loss='log', learning_rate='constant', eta0=0.0001, tol=0.0001) 
sgd_rs.fit(X_train, y_train)
sgd_rs_diag = model_diag(sgd_rs, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.668
Confusion matrix: 
[[0.99375 0.00625]
 [0.08471 0.00734]]


In [32]:
y_score_sgd_rs = sgd_rs.predict(X_test)
cm = confusion_matrix(y_test, y_score_sgd_rs)
print(cm)
sgd_test_diag_rs = model_oostest(sgd_rs,X_test, y_test)
sgd_test_auc_rs = plot_rocs([sgd_rs_diag, sgd_test_diag_rs], p_width=600, p_height=600, model_appendix=['SGD - 5folds','SGD - test'])
show(sgd_test_auc_rs)

[[14971   515]
 [   52     2]]
[[14971   515]
 [   52     2]]
AUC 0.722


### Random Forest Classifier

In [33]:
#random forest - benchmark
n_estimators = 200
max_leaf_nodes = 40
rf_clf = RandomForestClassifier(random_state=42,
                               n_estimators=n_estimators,
                               max_leaf_nodes=max_leaf_nodes,
                               class_weight="balanced",
                               n_jobs=7)

rf_clf.fit(X_train, y_train)
rf_clf_diag = model_diag(rf_clf, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.714
Confusion matrix: 
[[0.7649  0.2351 ]
 [0.03545 0.0566 ]]


In [34]:
y_score_rf = rf_clf.predict(X_test)
cm = confusion_matrix(y_test, y_score_rf)
print(cm)
rf_test_diag = model_oostest(rf_clf,X_test, y_test)
rf_test_auc = plot_rocs([rf_clf_diag, rf_test_diag], p_width=600, p_height=600, model_appendix=['RF - 5folds','RF - test'])
show(rf_test_auc)

[[12466  3020]
 [   35    19]]
[[12466  3020]
 [   35    19]]
AUC 0.671


In [35]:
# Number of trees in random forest
n_estimators = [150, 180, 200, 250, 280, 300, 350] 
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 10, 20]
# Maximum number of levels in tree
max_depth = [100,200, None]
#Max leaf nodes
max_leaf_nodes = [10,20,40,60]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
              'max_leaf_nodes':max_leaf_nodes}
print(random_grid)

{'n_estimators': [150, 180, 200, 250, 280, 300, 350], 'max_features': ['auto', 'sqrt', 10, 20], 'max_depth': [100, 200, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'max_leaf_nodes': [10, 20, 40, 60]}


In [36]:
rf = RandomForestClassifier() #base model
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 300, cv = 3, verbose=2, random_state=42, n_jobs =7)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 300 candidates, totalling 900 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:   54.5s
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed:  5.2min
[Parallel(n_jobs=7)]: Done 351 tasks      | elapsed: 12.0min
[Parallel(n_jobs=7)]: Done 634 tasks      | elapsed: 22.6min
[Parallel(n_jobs=7)]: Done 900 out of 900 | elapsed: 32.8min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=300, n_jobs=7,
          param_distributions={'n_estimators': [150, 180, 200, 250, 280, 300, 350], 'max_features': ['auto', 'sqrt', 10, 20], 'max_depth': [100, 200, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'max_leaf_nodes': [10, 20, 40, 60]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [37]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_leaf_nodes': 10,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': True}

In [38]:
#first randomized search_imp
rs1 = {'n_estimators': 280,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_leaf_nodes':60,
 'max_depth': 100,
 'bootstrap': True}

#second randomized search_imp
rs2 = {'n_estimators': 150,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_leaf_nodes': 60,
 'max_features': 20,
 'max_depth': 100,
 'bootstrap': False}

#third randomized search_p90
rs3 = {'n_estimators': 150,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_leaf_nodes': 60,
 'max_features': 20,
 'max_depth': 100,
 'bootstrap': False}

#fourth randomized search_p90_time
rs4 = {'n_estimators': 300,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_leaf_nodes': 10,
 'max_features': 'auto',
 'max_depth': 200,
 'bootstrap': True}

#seventh randomized search_p180_time
rs7 = {'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_leaf_nodes': 10,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': True}

In [39]:
#random_forest from rand
n_estimators=rs7['n_estimators']
min_samples_split=rs7['min_samples_split']
min_samples_leaf=rs7['min_samples_leaf']
max_leaf_nodes=rs7['max_leaf_nodes']
max_features=rs7['max_features']
max_depth=rs7['max_depth']
bootstrap=rs7['bootstrap']




rf_rs = RandomForestClassifier(random_state=42,
                               n_estimators=n_estimators,
                                max_features=max_features,
                                min_samples_split=min_samples_split,
                               min_samples_leaf=min_samples_leaf,
                               class_weight="balanced", bootstrap=bootstrap,
                               n_jobs=7)
rf_rs.fit(X_train, y_train)
rf_rs_diag = model_diag(rf_rs, X_train, y_train, run_confusion_matrix=True, CrossValFolds=5)

AUC 0.697
Confusion matrix: 
[[0.91415 0.08585]
 [0.05977 0.03228]]


In [40]:
y_score_rf_rs = rf_rs.predict(X_test)
cm = confusion_matrix(y_test, y_score_rf_rs)
print(cm)
rf_rs_test_diag = model_oostest(rf_rs,X_test, y_test)
rf_rs_test_auc = plot_rocs([rf_rs_diag, rf_rs_test_diag], p_width=600, p_height=600, model_appendix=['RF - 5folds','RF - test'])
show(rf_rs_test_auc)

[[14879   607]
 [   36    18]]
[[14879   607]
 [   36    18]]
AUC 0.866


In [41]:
#save_sk_model(rf_rs, "../data/models/", 'rf_p180_time', 'benchmark')

Saving model to ../data/models/benchmark_rf_p180_time_190710_1337.pkl


### Training Performance Comparison

In [42]:
rf_sgd_roc = plot_rocs([sgd_clf_diag, rf_rs_diag], p_width=600, p_height=600, model_appendix=['SGD', 'Random Forest'])
show(rf_sgd_roc)

### Test Performance Comparison

In [43]:
rf_sgd_roc = plot_rocs([sgd_test_diag, rf_rs_test_diag], p_width=600, p_height=600, model_appendix=['SGD', 'Random Forest'])
show(rf_sgd_roc)